In [1]:
print("hellp")

hellp


In [11]:
from dotenv import load_dotenv
from pymongo import MongoClient
from openai import OpenAI
import uuid
load_dotenv()
openai = OpenAI()

In [4]:
prompt = "who is the president of india ?"

response=openai.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "user", "content": prompt},
    ],
    temperature=0.7,
    max_tokens=100
)

In [10]:
data=response.choices[0].message.content

In [21]:
try:
    conn=MongoClient()
    print("Mongo connection created")
except Exception as e:
    print(f"An error occurred: {e}")

Mongo connection created


In [22]:
# database
db=conn.database


In [17]:
collection = db.my_gfg_collection

In [23]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'database'), 'my_gfg_collection')

In [24]:
response_1 = {
    "ID":uuid.uuid4(),
    "Question": prompt,
    "Answer": data
}

rec_1=collection.insert_one(response_1)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 671cc2d243ab99dd47510191, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [12]:
import os
import json
import uuid

In [37]:
session_id = str(uuid.uuid4())

# Define the user input data to be appended
prompt = "hello"  # Replace with actual prompt as needed
additional_info = "hello world python"  # Replace with actual additional info

new_data = {
    "session_id": session_id,
    "user_input": {
        "question": prompt,
        "additional_info": additional_info
    }
}


folder_name = "AIResponse"
file_name = "response.json"
file = os.path.join(folder_name, file_name)

# Check if directory exists, if not create it
os.makedirs(folder_name,exist_ok=True)


if os.path.exists(file):
    with open(file, "r") as f:
        data = json.load(f)
else:
    data = []

data.append(new_data)

# Write the updated list back to the JSON file
with open(file, 'w') as f:
    json.dump(data, f , indent=4)

In [34]:
import os
import json
import uuid

# Define the folder and file names
folder_name = "AIResponse"
file_name = "responses.json"
file_path = os.path.join(folder_name, file_name)

# Ensure the folder exists
os.makedirs(folder_name, exist_ok=True)

# Generate a new unique session ID
session_id = str(uuid.uuid4())

# Define the user input data to be appended
prompt = "hello"  # Replace with actual prompt as needed
additional_info = "hello world python"  # Replace with actual additional info

new_data = {
    "session_id": session_id,
    "user_input": {
        "question": prompt,
        "additional_info": additional_info
    }
}

# Load existing data from the file, or initialize an empty list if the file doesn't exist
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
else:
    data = []

# Append the new data to the existing list
data.append(new_data)

# Write the updated list back to the JSON file
with open(file_path, 'w') as f:
    json.dump(data, f, indent=4)

print(f"New session added to '{file_path}' with session_id: {session_id}")


New session added to 'AIResponse\responses.json' with session_id: 312fe89a-57fe-4148-bc32-3c922188569a


In [2]:
# Chat History with version 1 document


In [12]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os

llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_chroma import Chroma

In [18]:
def loadchromadb():
        vector_store = Chroma(persist_directory="../ChormaDB",embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))

        retriever = vector_store.as_retriever(k=3)

        return retriever

In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever = loadchromadb()

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [20]:
chat_history = []

question = " Explain about this Millennia Credit Card card and offer details ?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})


In [21]:
ai_msg_1

{'input': ' Explain about this Millennia Credit Card card and offer details ?',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://www.hdfcbank.com/personal/pay/cards/credit-cards'}, page_content='10X CashPoints on Amazon, BigBasket, Flipkart, Reliance Smart SuperStore & Swiggy\n\n5X CashPoints on EMI spends at merchant locations\n\n2 CashPoints on all other spends ( Excluding fuel, Wallets loads )\n\nGift vouchers worth upto ₹2000 annually\n\nAPPLY ONLINE APPLY ONLINE KNOW MORE KNOW MORE\n\nMillennia Credit Card\n\nMillennia Credit Card\n\nThe Best CashBack Credit Card\n\n5% Cashback on Amazon, BookMyShow, Cult.fit, Flipkart, Myntra, Sony LIV, Swiggy, Tata CLiQ, Uber and Zomato\n\n1% cashback on all other spends (except Fuel) including EMI & Wallet transactions\n\n₹1000 worth gift vouchers on spends of ₹1,00,000 and above in each calendar quarter\n\nAPPLY ONLINE APPLY ONLINE Know more Know more\n\nFreedom Credit Card\n\nFreedom Credit CardLifestyle\n\nThe Best Car

In [22]:
question = " Explain about more please "
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})


In [23]:
ai_msg_1

{'input': ' Explain about more please ',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://www.hdfcbank.com/personal/pay/cards/credit-cards'}, page_content='Know More Know More\n\nLifestyle\n\nLifestyle\n\nEnjoy a life more rewarding with inspired choices and luxurious indulgences that are ready for you.\n\nfalse\n\nRegalia Gold Credit Card\n\nRegalia Gold Credit CardSuper Premium\n\nExperience the Golden Side Of Life\n\nComplimentary Club Vistara Silver Tier and MMT Black Elite membership as Welcome benefits.\n\n₹ 1,500 worth vouchers on quarterly spends of Rs. 1.5 lakh\n\n₹ 5,000 worth flight vouchers on annual spends of Rs. 5 lakh and Rs. 7.5 lakhs.\n\n5X Reward Points on spends at Marks & Spencer, Myntra, Nykaa & Reliance Digital!\n\nApply Online Apply Online KNOW MORE KNOW MORE\n\nMillennia Credit Card\n\nMillennia Credit Card\n\nThe Best CashBack Credit Card\n\n5% Cashback on Amazon, BookMyShow, Cult.fit, Flipkart, Myntra, Sony LIV, Swiggy, Tata CLiQ, Uber a

In [25]:
print(ai_msg_1['answer'])

The context provided contains information about two credit cards offered by ICICI Bank: the Regalia Gold Credit Card and the Millennia Credit Card. The Regalia Gold Credit Card offers benefits like complimentary Club Vistara Silver Tier membership, flight vouchers, and 5X reward points on specific categories. On the other hand, the Millennia Credit Card is a cashback credit card that provides 5% cashback on select merchants, 1% cashback on other spends, and gift vouchers based on quarterly spends.


In [26]:
question = " Explain clearly beacuse i cant get it "
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

In [27]:
ai_msg_1

{'input': ' Explain clearly beacuse i cant get it ',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://www.icicibank.com/personal-banking/cards/credit-card'}, page_content='A Credit Card is a payment card that allows you to make purchases without immediate payment. It is widely accepted and you need not carry cash with you always. Your bank issues this card based on your income and other eligibility criteria. They set a limit on how much you can spend. However, you have to pay back the amount you spend within a specific time. If you are late in making the payment, you will have to pay the interest along with some additional charges.\n\nBenefits of a credit card are:\n\nUniversal payment acceptance\n\nEarn reward points or cashback for every retail spend using ICICI Bank Credit Card\n\nEnjoy credit free payments up to a maximum of <48> days including the due date\n\nFuel surcharge waiver\n\nExciting discount offers on dining out and movie tickets\n\nComplimentary l

In [1]:
#MongoDB Connection String : mongodb://localhost:27017/

from langchain_mongodb.chat_message_histories import MongoDBChatMessageHistory

chat_message_history = MongoDBChatMessageHistory(
    session_id = "demo_session_id",
    connection_string = "mongodb://localhost:27017/",
    database_name = "AIdemo",
    collection_name = "chat_histories" 
)

# Add a new chat message to the history

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hello How can i help you?")
    

In [2]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello How can i help you?', additional_kwargs={}, response_metadata={})]

In [11]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

prompt = ChatPromptTemplate.from_messages(
    [
        (
        "system","You are an helpfull assistent answer the user question.if you dont know the answer please ask user to provide more information ,dont make anything more complicated"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{question}"),
    ]
)

chain = prompt | llm



In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id :MongoDBChatMessageHistory(
        session_id = session_id,
        connection_string = "mongodb://localhost:27017/",
        database_name = "AIdemo",
        collection_name = "chat_histories"  # replace with your collection name
    ),
    input_messages_key="question",
    history_messages_key="chat_history"

)



In [13]:
chain_with_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function <lambda> at 0x000001DAC757C310>, input_messages_key='question', history_messages_key='chat_history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [14]:
config = {"configurable" : {"session_id":"demo_session_id"}}


user_prompt = "How to speak english with in 30 days?"
# if you want to get instant response from ai user stream instead of invoke
chain_with_history.invoke({"question" : user_prompt},config = config)

AIMessage(content="Improving English speaking skills in 30 days is a challenging goal, but it is possible with dedication and consistent practice. Here are some tips to help you achieve this:\n\n1. Practice speaking every day: Make an effort to speak in English every day, even if it's just for a few minutes. This will help you become more comfortable with the language.\n\n2. Find a language partner: Practice speaking with a friend, family member, or language exchange partner who is fluent in English. This will give you the opportunity to practice speaking in a natural, conversational setting.\n\n3. Watch English movies and TV shows: Watching English-language content can help you improve your listening and speaking skills. Pay attention to how native speakers pronounce words and phrases.\n\n4. Use language learning apps: There are many apps available that can help you improve your English speaking skills, such as Duolingo, Babbel, and Rosetta Stone.\n\n5. Take a speaking course: Conside